# RAG Avancé: HQ (Hypothetical Question),Chromadb, BM25, et Gemini 

#### Bibliothèques

In [153]:
from langchain_community.document_loaders import PyPDFLoader
from pydantic import BaseModel
from typing import List, Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from rank_bm25 import BM25Okapi
from langchain.llms import OpenAI
import os

load_dotenv()

#API key:
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

True

### Étape 1 : Définir la structure de données avec Pydantic

In [154]:
class DocumentData(BaseModel):
    title: Optional[str]
    author: Optional[str]
    content: str
    metadata: Optional[dict]
    source: str
    page_number: int
    excerpt: str

#### Étape 2 : Charger et extraire les données du/des PDF(s) 

In [155]:
loader = PyPDFLoader("data/doc.pdf")
data = loader.load()

In [156]:
# extrait de texte autour de la réponse
def get_excerpt(page_content: str, query: str, window_size: int = 100):
    """Retourne un extrait de texte autour de la question"""
    idx = page_content.find(query)
    if idx == -1:
        return page_content[:window_size] #
    start_idx = max(idx - window_size, 0)
    end_idx = min(idx + len(query) + window_size, len(page_content))
    return page_content[start_idx:end_idx]

In [157]:
# Collecte des données structurées pour chaque page du PDF
documents_data = []
for page_number, page in enumerate(data, 1):
    document_content = page.page_content
    excerpt = get_excerpt(document_content, query) 
    metadata = {"source": "doc.pdf", "page": page_number}

    document_data = DocumentData(
        title="Exemple de Titre", 
        author="Nom de l'Auteur", 
        content=document_content,
        metadata=metadata,
        source="doc.pdf", 
        page_number=page_number,
        excerpt=excerpt 
    )
    documents_data.append(document_data)

#### Split & Embedding

In [158]:
# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(data)

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("chromadb")

#vector

vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
vector[:5]

[0.0031878091394901276,
 -0.03396224603056908,
 -0.04738517105579376,
 -0.02076413296163082,
 0.04006507992744446]

#### HQ & Retrieval

In [159]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("comment se passe la fourniture d’un complément de liquide de refroidissement ")

#ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

#BM25
bm25_retriever = BM25Retriever.from_documents(docs)
# Création de l'EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vectorstore.as_retriever()],
    weights=[0.5, 0.5]
)

#Fonction questions hypothétiques
def generate_hypothetical_questions(query, llm):
    prompt = f"Générez 3 questions hypothétiques liées à cette requête : {query}"
    response = llm.invoke(prompt)
    return response.content.split('\n')

#### Prompt engineering

In [160]:
system_prompt = (
    '''Tu es un assistant technique ton ôle est de répondre aux questions.
    Utilise les éléments de contexte suivants pour répondre aux questions
    Si tu ne connais pas la réponse, dites je ne connais pas. Utilise trois
    phrases au maximum et faites en sorte que la réponse soit concise.'''

    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


#### Chaine de RAG avancée

In [161]:

def advanced_rag_chain(query):
    hq_questions = generate_hypothetical_questions(query, llm)  # Génération de questions hypothétiques
    
    all_docs = ensemble_retriever.invoke(query)  # Recherche avec la requête originale et les questions hypothétiques
    for hq in hq_questions:
        all_docs.extend(ensemble_retriever.invoke(hq))
    
    # Déduplication et limitation du nombre de documents
    unique_docs = list({doc.metadata['source'] + str(doc.metadata['page']): doc for doc in all_docs}.values())
    context_docs = unique_docs[:10] 

    # Génération de la réponse
    response = rag_chain.invoke({"input": query, "context": context_docs})

    # Extraction des informations de source
    if context_docs:
        source_info = context_docs[0].metadata
        source_name = source_info.get('source', 'N/A')
        page_number = source_info.get('page', 'N/A')
        excerpt = context_docs[0].page_content[:200] + "..."

        # Ajout des informations sur la source
        source_string = f"Source: {source_name}\nPage: {page_number}\nExtrait: {excerpt}"
    else:
        source_string = "Aucune source trouvée."

    # Affichage des questions hypothétiques
    hq_display = "\n".join(hq_questions)
    output = f"Questions hypothétiques générées :\n{hq_display}\n\nRéponse : {response['answer']}\n\n{source_string}"
    
    return output


In [162]:
# Utilisation
query = "Pour les Matériels Roulants autres que les wagons, en quoi consiste les opérations de maintenance corrective?"
result = advanced_rag_chain(query)
print(result)

Questions hypothétiques générées :
1. Si un système de climatisation d'une locomotive tombe en panne en plein été, quelles opérations de maintenance corrective seraient entreprises pour remettre rapidement le système en état de fonctionnement et minimiser l'impact sur le service?

2. Imaginons une fissure détectée sur le châssis d'une automotrice.  Quelles seraient les étapes de la maintenance corrective, de l'évaluation des dommages à la réparation finale, et comment s'assurerait-on de la sécurité du matériel roulant après l'intervention?

3.  En cas de défaillance soudaine du système de freinage d'un tramway en pleine ligne, quelles procédures de maintenance corrective seraient mises en place en urgence pour garantir la sécurité des passagers et la reprise rapide du service?

Réponse : Pour le matériel roulant autre que les wagons, la maintenance corrective peut inclure des travaux suite à des déprédations intérieures/extérieures, des opérations saisonnières (climatisation, chauffage